### Import Libraries

In [1]:
# tensorflow
import tensorflow as tf
from tensorflow.keras import models, layers, callbacks



### Data setup

### NN Setup

In [2]:
# DeepNN model
n_features = 5
model = models.Sequential(name="DeepNN", layers=[
    ### hidden layer 1
    layers.Dense(name="h1", input_dim=n_features,
                 units=int(round((n_features+1)/2)), 
                 activation='relu'),
    layers.Dropout(name="drop1", rate=0.2),
    
    ### hidden layer 2
    layers.Dense(name="h2", units=int(round((n_features+1)/4)), 
                 activation='relu'),
    layers.Dropout(name="drop2", rate=0.2),
    
    ### layer output
    layers.Dense(name="output", units=1, activation='sigmoid')
])
model.summary()

Model: "DeepNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 h1 (Dense)                  (None, 3)                 18        
                                                                 
 drop1 (Dropout)             (None, 3)                 0         
                                                                 
 h2 (Dense)                  (None, 2)                 8         
                                                                 
 drop2 (Dropout)             (None, 2)                 0         
                                                                 
 output (Dense)              (None, 1)                 3         
                                                                 
Total params: 29
Trainable params: 29
Non-trainable params: 0
_________________________________________________________________
